# Object Matching Evaluation for Filtered Images

## Introduction

...

...

...

## Background

...

...

...

## Implementation

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from sklearn.metrics.pairwise import cosine_similarity

from expt_utils import *

### Compare Original Image to Desired Image

In [14]:
img = cv.imread(f'{DS_DIR}/frames/train/00029/flip/00029_1640_flip.jpg')
img_desired = cv.imread(f'{IMG_DIR}/desired_mask_for_00029_1640_flip.png', cv.IMREAD_GRAYSCALE)

cv.imshow('Original Image', img)
cv.imshow('Desired Image', img_desired)

destroy_when_esc()

### Apply Some Filters

In [3]:
img_canny = img_to_canny_edges(img)
img_mexican_hat = img_to_mexican_hat(img).astype('uint8')
img_hanny = img_to_hanny(img).astype('uint8')
img_blur = cv.GaussianBlur(img, (3, 3), 0).astype('uint8')

cv.imshow('Image - Canny Edges Filter', img_canny)
cv.imshow('Image - Mexican Hat Filter', img_mexican_hat)
cv.imshow('Image - Hanny Filter', img_hanny)
cv.imshow('Image - Gaussian Blur Filter', img_blur)

destroy_when_esc()

In [4]:
img_mexican_hat_thresh = imge_otsu_thresh(img_mexican_hat)
img_hanny_thresh = imge_otsu_thresh(img_hanny)
img_blur_thresh = imge_otsu_thresh(img_blur)

cv.imshow('Thresh Image - Mexican Hat Filter', img_mexican_hat_thresh)
cv.imshow('Thresh Image - Hanny Fsilter', img_hanny_thresh)
cv.imshow('Thresh Image - Gaussian Blur Filter', img_blur_thresh)

destroy_when_esc()

### Compare and Evaluate the Results

#### Correlation Coeficient Method

In [6]:
corrcof_canny = np.corrcoef(img_canny.flatten(), img_desired.flatten())[0, 1]
corrcof_mexican_hat = np.corrcoef(img_mexican_hat_thresh.flatten(), img_desired.flatten())[0, 1]
corrcof_hanny = np.corrcoef(img_hanny_thresh.flatten(), img_desired.flatten())[0, 1]
corrcof_gaussian = np.corrcoef(img_blur_thresh.flatten(), img_desired.flatten())[0, 1]

print(corrcof_canny, corrcof_mexican_hat, corrcof_hanny, corrcof_gaussian)

0.10836446196295399 0.1029462377018336 0.012908514275302999 0.005201106664143005


#### Eculodian Distance Method

In [9]:
img_canny_diff = np.sqrt((img_canny - img_desired)**2)
img_mexican_hat_diff = np.sqrt((img_mexican_hat_thresh - img_desired)**2)
img_hanny_diff = np.sqrt((img_hanny_thresh - img_desired)**2)
img_blur_diff = np.sqrt((img_blur_thresh - img_desired)**2)

dis_ecl_canny = np.sum(img_canny_diff)
dis_ecl_mexican_hat = np.sum(img_mexican_hat_diff)
dis_ecl_hanny = np.sum(img_hanny_diff)
dis_ecl_blur = np.sum(img_blur_diff)

dis_ecl_canny = np.floor(dis_ecl_canny)
dis_ecl_mexican_hat = np.floor(dis_ecl_mexican_hat)
dis_ecl_hanny = np.floor(dis_ecl_hanny)
dis_ecl_blur = np.floor(dis_ecl_blur)

print(dis_ecl_canny,dis_ecl_mexican_hat,dis_ecl_hanny,dis_ecl_blur)

16880.0 inf inf 53340.0


#### Cosine Similarity Method

In [10]:
img_canny_pixels = img_canny.flatten().reshape(1,-1)
img_mexican_hat_pixels = img_mexican_hat_thresh.flatten().reshape(1,-1)
img_hanny_pixels = img_hanny_thresh.flatten().reshape(1,-1)
img_blur_pixels = img_blur_thresh.flatten().reshape(1,-1)
img_desired_pixels = img_desired.flatten().reshape(1,-1)

canny_cos_scr = cosine_similarity(img_canny_pixels, img_desired_pixels)[0][0]
mexican_hat_cos_scr = cosine_similarity(img_mexican_hat_pixels, img_desired_pixels)[0][0]
hanny_cos_scr = cosine_similarity(img_hanny_pixels, img_desired_pixels)[0][0]
blur_cos_scr = cosine_similarity(img_blur_pixels, img_desired_pixels)[0][0]

print(canny_cos_scr,mexican_hat_cos_scr,hanny_cos_scr,blur_cos_scr)

0.12909676558052946 0.1616295409064753 0.10956491648337013 0.056964859934944795


### ???

In [13]:
# Perform template matching
result = cv.matchTemplate(img_canny, img_desired, cv.TM_CCOEFF_NORMED)

# Define a threshold to consider a match
threshold = 0.01
locations = np.where(result >= threshold)

# Draw bounding boxes around the detected objects
for pt in zip(*locations[::-1]):
    print(1)
    bottom_right = (pt[0] + img_desired.shape[1], pt[1] + img_desired.shape[0])
    cv.rectangle(img_mexican_hat_thresh, pt, bottom_right, 255, 2)

# Display the result
cv.imshow('Detected Objects', img_mexican_hat_thresh)
cv.waitKey(0)
cv.destroyAllWindows()

1


## Conclusion

...

...

...